<!-- NOTEBOOK_METADATA source: "⚠️ Jupyter Notebook" title: "Cookbook - Langchain Integration (JS/TS)" sidebarTitle: "Langchain Integration (JS/TS)" description: "Cookbook with examples of the Langfuse Integration for Langchain (JS/TS)." category: "Examples" -->

# Cookbook: Langchain Integration (JS/TS)

<a href="/guides/cookbook/integration_langchain"><img className="inline" alt="Python" src="https://img.shields.io/badge/Python-3776AB?style=flat&logo=python&logoColor=white" /></a> <a href="/guides/cookbook/js_integration_langchain"><img className="inline" alt="JS/TS" src="https://img.shields.io/badge/JS/TS-d4d4d8?style=flat&logo=javascript&logoColor=white" /></a>

This is a cookbook with examples of the Langfuse integration for Langchain (JS/TS).

Follow the [integration guide](https://langfuse.com/integrations/frameworks/langchain) to add this integration to your Langchain project.

<!-- STEPS_START -->
## Set Up Environment

Get your Langfuse API keys by signing up for [Langfuse Cloud](https://cloud.langfuse.com/) or [self-hosting Langfuse](https://langfuse.com/self-hosting). You’ll also need your OpenAI API key.

> **Note**: This cookbook uses **Deno.js** for execution, which requires different syntax for importing packages and setting environment variables. For Node.js applications, the setup process is similar but uses standard `npm` packages and `process.env`.

In [ ]:
// Langfuse authentication keys
Deno.env.set("LANGFUSE_PUBLIC_KEY", "pk-lf-***");
Deno.env.set("LANGFUSE_SECRET_KEY", "sk-lf-***");

// Langfuse host configuration
// For US data region, set this to "https://us.cloud.langfuse.com"
Deno.env.set("LANGFUSE_BASE_URL", "https://cloud.langfuse.com")

// Set environment variables using Deno-specific syntax
Deno.env.set("OPENAI_API_KEY", "sk-proj-***");

With the environment variables set, we can now initialize the `langfuseSpanProcessor` which is passed to the main OpenTelemetry SDK that orchestrates tracing.

In [ ]:
// Import required dependencies
import 'npm:dotenv/config';
import { NodeSDK } from "npm:@opentelemetry/sdk-node";
import { LangfuseSpanProcessor } from "npm:@langfuse/otel";
 
// Export the processor to be able to flush it later
// This is important for ensuring all spans are sent to Langfuse
export const langfuseSpanProcessor = new LangfuseSpanProcessor({
    publicKey: process.env.LANGFUSE_PUBLIC_KEY!,
    secretKey: process.env.LANGFUSE_SECRET_KEY!,
    baseUrl: process.env.LANGFUSE_BASE_URL ?? 'https://cloud.langfuse.com', // Default to cloud if not specified
    environment: process.env.NODE_ENV ?? 'development', // Default to development if not specified
  });
 
// Initialize the OpenTelemetry SDK with our Langfuse processor
const sdk = new NodeSDK({
  spanProcessors: [langfuseSpanProcessor],
});
 
// Start the SDK to begin collecting telemetry
// The warning about crypto module is expected in Deno and doesn't affect basic tracing functionality. Media upload features will be disabled, but all core tracing works normally
sdk.start();

## Step 2: Instantiate the CallbackHandler

Instantiate the CallbackHandler and pass it to your chain’s `.invoke()` or `.stream()` method in the callbacks array. All operations within the chain will be traced as nested observations.

In [ ]:
import { CallbackHandler } from "npm:@langfuse/langchain";
 
const langfuseHandler = new CallbackHandler({
  sessionId: "user-session-123",
  userId: "user-abc",
  tags: ["langchain-test"],
});

## Step 3: Langchain Example

We create a simple agent that can answer questions and call tools. The agent will use OpenAI GPT-5-mini as its language model, a basic weather function as a tool, and a simple prompt to guide its behavior. All tools and model calls are traced in Langfuse.

In [ ]:
import { createAgent, tool } from "npm:langchain";
import * as z from "npm:zod";

const getWeather = tool(
  (input) => `It's always sunny in ${input.city}!`,
  {
    name: "get_weather",
    description: "Get the weather for a given city",
    schema: z.object({
      city: z.string().describe("The city to get the weather for"),
    }),
  }
);

const agent = createAgent({
  model: "openai:gpt-5-mini",
  tools: [getWeather],
});

In [ ]:
console.log(
    await agent.invoke(
        { messages: [{ role: "user", content: "What's the weather in San Francisco?" }] }, 
        { callbacks: [langfuseHandler] }
    )
);

## Step 4: Explore the trace in Langfuse

In the Langfuse interface, you can see a detailed trace of all steps in the Langchain application.

![Langfuse Trace](https://langfuse.com/images/cookbook/example-js-sdk/js_integration_langchain_trace.png)

[Public trace in the Langfuse UI](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/2c1581dd9cecdafb6ca091b83d7ea99a?timestamp=2025-10-30T08:45:41.319Z)
<!-- STEPS_END -->